In [2]:
import json
import uuid
from cryptography.fernet import Fernet

# Generate a unique decentralized identifier (DID)
def generate_did():
    return f"did:example:{uuid.uuid4()}"

# Encrypt data to simulate cryptographic proof
def encrypt_data(data, key):
    f = Fernet(key)
    encrypted_data = f.encrypt(data.encode())
    return encrypted_data

# Decrypt data
def decrypt_data(encrypted_data, key):
    f = Fernet(key)
    decrypted_data = f.decrypt(encrypted_data).decode()
    return decrypted_data

# Create a verifiable credential
def create_verifiable_credential(student_info, issuer, key):
    credential = {
        "context": "https://www.w3.org/2018/credentials/v1",
        "type": ["VerifiableCredential"],
        "issuer": issuer,
        "credentialSubject": student_info,
    }
    serialized_credential = json.dumps(credential, indent=2)
    encrypted_credential = encrypt_data(serialized_credential, key)
    return encrypted_credential

# Main function to create a DID and VC
def main():
    # Generate cryptographic key
    key = Fernet.generate_key()
    
    # University and student info
    university_did = generate_did()
    student_info = {
        "id": generate_did(),
        "name": "John Doe",
        "studentId": "UNINotts-123456",
        "course": "Computer Science"
    }

    # Create the VC
    encrypted_vc = create_verifiable_credential(student_info, university_did, key)
    print("Encrypted Verifiable Credential:")
    print(encrypted_vc)
    
    # Decrypting the VC for display
    decrypted_vc = decrypt_data(encrypted_vc, key)
    print("\nDecrypted Verifiable Credential:")
    print(decrypted_vc)

if __name__ == "__main__":
    main()


Encrypted Verifiable Credential:
b'gAAAAABmMqihpTbxHi_EeOI7pyn17qaf2EKyTD0H74Qh42mGGhYzkl3VXluMEb1kngO3S8F2H8N7oisUEZvaLEo5ooUs0uK8Ikt5Q9rUdHzcsW5wWQLJo6278JpnMKP2xEjZuDLGo-q2SVF5lTbjGoDpngK_EeOGPTcC5IpgD0n2_ADocaJ3jhjXqUjSeMgaPg25MzflIGGUp7VG46gSpK2OmYNsCGczAKyFAjImKOBFzMTCkxRf3I2dj6QJLogbvGCJacm0QewX-jpZnTCLhjHarqqDzVRm_NYitfMUxhB673awMqA8eklzMTzDQqxCzg7TO-LpDAr1TzTxXiUfgRqKWhibJOmnkc8IUMH66ElsoUPKgvw7j4kIVS18PyheoLLCXFaovSc6ZzxE-d_FzPvHczxE_m2JZ9sFwK5YTUv2MTI07sr_MFpq_ejJjUA3-avRo7JfgPa1FYIFbK8mC8vyMIFYO_j-tglInWkphLd9shdvYXdU0Jyn8fKGM8EVhHZ1c2wiLrLYxAkmXs662OmhKefvfT-SQQ=='

Decrypted Verifiable Credential:
{
  "context": "https://www.w3.org/2018/credentials/v1",
  "type": [
    "VerifiableCredential"
  ],
  "issuer": "did:example:1840be00-ed60-4c74-906c-1e4261e83b99",
  "credentialSubject": {
    "id": "did:example:4878adeb-3157-4654-902b-d0ef5ff0e682",
    "name": "John Doe",
    "studentId": "UNINotts-123456",
    "course": "Computer Science"
  }
}


In [3]:
# !pip install qrcode

In [30]:
import json
import uuid
import qrcode
from cryptography.fernet import Fernet

# Generate a unique decentralized identifier (DID)
def generate_did():
    return f"did:example:{uuid.uuid4()}"

# Encrypt data to simulate cryptographic proof
def encrypt_data(data, key):
    f = Fernet(key)
    encrypted_data = f.encrypt(data.encode())
    return encrypted_data

# Decrypt data
def decrypt_data(encrypted_data, key):
    f = Fernet(key)
    decrypted_data = f.decrypt(encrypted_data).decode()
    return decrypted_data

# Create a verifiable credential
def create_verifiable_credential(student_info, issuer, key):
    credential = {
        "context": "https://www.w3.org/2018/credentials/v1",
        "type": ["VerifiableCredential"],
        "issuer": issuer,
        "credentialSubject": student_info,
    }
    serialized_credential = json.dumps(credential, indent=2)
    encrypted_credential = encrypt_data(serialized_credential, key)
    return encrypted_credential

# Generate QR Code for any text
def generate_qr_code(data):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save("did_qr_code.png")  # Saving the QR code to a file
    return "QR code generated and saved as did_qr_code.png"

# Main function to create a DID and VC
def main():
    # Generate cryptographic key
    key = Fernet.generate_key()
    
    # University and student info
    university_did = generate_did()
    student_info = {
        "id": generate_did(),
        "name": "John Doe",
        "studentId": "UNINotts-123456",
        "course": "Computer Science"
    }

    # Create the VC
    encrypted_vc = create_verifiable_credential(student_info, university_did, key)
    print("Encrypted Verifiable Credential:")
    print(encrypted_vc)
    
    # Decrypting the VC for display
    decrypted_vc = decrypt_data(encrypted_vc, key)
    print("\nDecrypted Verifiable Credential:")
    print(decrypted_vc) 

    # Generate QR Code for the DID
    qr_code_message = generate_qr_code(json.dumps(student_info))
    print(qr_code_message)

if __name__ == "__main__":
    main()


Encrypted Verifiable Credential:
b'gAAAAABmMqpoiZIvsQIGryBezcnEn0WKLdv38B2TPSy7QHtJqezTHvxBzvPq49wWMFXUIMQNNf0FKHrsg2ZGYNzHOGlMseBv0gdDXWNS96Sib6wlIOWg5vvwaaaNOED1ZzxooY50kfpeKFLuMqkc0E2vi_hfzKsbJtR6E0NPjw-ZLJcVGMh4MrU1JGCJ4otT9fVsSUFT3sjNjZNnvA5WS_NSTDLL2QgTIpZ--buQaVz-YP5c6V3H-E42IwZDaNQiS41r9y01SaIu-vSam3qY6Rq2BMm4BDsQKDI48eRzy8J42a0hHSz5K7C2QwXdO6OLtPxiv3jJihalBj7nAscHs8qPderBUhomi-d0TlTYGQ7GBxPUDNDsA4jct6__nqaRjzptyFr3ygGGTi4iPLEHGNvFPAE60TwrFwHaxzKzP6V172BAQ_okVbkATog3Lh-hlJxb4yGeSvRrvydCFFsvXfDYDVQWzL-oFMAMqDMBJj4QWsqZLz-G4GxmqjK-Yvxq2oacaliQCliYFUM5yW7FDsP9BO0EIACe8w=='

Decrypted Verifiable Credential:
{
  "context": "https://www.w3.org/2018/credentials/v1",
  "type": [
    "VerifiableCredential"
  ],
  "issuer": "did:example:bbe7131f-b73b-4999-9a06-7727b0521397",
  "credentialSubject": {
    "id": "did:example:6536ae2f-49d7-4200-8aa9-2a9fc1c7226e",
    "name": "John Doe",
    "studentId": "UNINotts-123456",
    "course": "Computer Science"
  }
}
QR code generated and saved as

In [31]:
# Generate cryptographic key
key = Fernet.generate_key()

# University and student info
university_did = generate_did()
student_info = {
    "id": generate_did(),
    "name": "John Doe",
    "studentId": "UNINotts-123456",
    "course": "Computer Science"
}

# Create the VC
encrypted_vc = create_verifiable_credential(student_info, university_did, key)
print("Encrypted Verifiable Credential:")
print(encrypted_vc)

# Decrypting the VC for display
decrypted_vc = decrypt_data(encrypted_vc, key)
print("\nDecrypted Verifiable Credential:")
print(decrypted_vc) 

Encrypted Verifiable Credential:
b'gAAAAABmMqpy8DUgodmpXdWgZLRc92hCzR1P6I-5qqw3mc9Ck74qlo4F-4uvM_77D3NDUvEh41zGIG4GHM8iAy7fVO1VObg7CkETY604Y7w8XB9Zb4bLfqIgZ9kyFSUXNqRL91F8zPJY81v9goyBjUm1_6Ya37XjoRJf0bDFYcSS-rnXAlshUklPhvKn3kAt883O9anuAw9cM8pL88JsAOVDC7FKHnMVfwZiysSQp6FN-6JL6i2YxOULfzYq1B3amWQoxFc_3jdpDhgFE582xBWDdzw-bRw9D1HMsygLOzGU5hZUe_FC_kird5bERASEBBjMsY9VjsjTSbzE94sKZO9YFW-EQ3cIJPhyeYt21RFcr0B01onlqlFUYrU7SGlLI5i_dNHzIcHz_1TNstqSVRVFS98_BHOPgySw9wIkYW09Io9z9DiNvEx6kXc7gzr_RKSeFQ37309YOp-s6h8BFD3uSkLWWr5873_1pCQ6p1lxHw6EW8E1WmsdLVcNjCEUE-j6FbH-J6RHWRdJa-DEMQTa_f0lcEe03A=='

Decrypted Verifiable Credential:
{
  "context": "https://www.w3.org/2018/credentials/v1",
  "type": [
    "VerifiableCredential"
  ],
  "issuer": "did:example:975de2a3-e0e6-4775-9ea2-d758fe3a2277",
  "credentialSubject": {
    "id": "did:example:496d8f01-14bc-440d-b7cf-00e9895c01ef",
    "name": "John Doe",
    "studentId": "UNINotts-123456",
    "course": "Computer Science"
  }
}


In [32]:
encrypted_vc

b'gAAAAABmMqpy8DUgodmpXdWgZLRc92hCzR1P6I-5qqw3mc9Ck74qlo4F-4uvM_77D3NDUvEh41zGIG4GHM8iAy7fVO1VObg7CkETY604Y7w8XB9Zb4bLfqIgZ9kyFSUXNqRL91F8zPJY81v9goyBjUm1_6Ya37XjoRJf0bDFYcSS-rnXAlshUklPhvKn3kAt883O9anuAw9cM8pL88JsAOVDC7FKHnMVfwZiysSQp6FN-6JL6i2YxOULfzYq1B3amWQoxFc_3jdpDhgFE582xBWDdzw-bRw9D1HMsygLOzGU5hZUe_FC_kird5bERASEBBjMsY9VjsjTSbzE94sKZO9YFW-EQ3cIJPhyeYt21RFcr0B01onlqlFUYrU7SGlLI5i_dNHzIcHz_1TNstqSVRVFS98_BHOPgySw9wIkYW09Io9z9DiNvEx6kXc7gzr_RKSeFQ37309YOp-s6h8BFD3uSkLWWr5873_1pCQ6p1lxHw6EW8E1WmsdLVcNjCEUE-j6FbH-J6RHWRdJa-DEMQTa_f0lcEe03A=='

In [33]:
test = json.dumps(student_info)

In [34]:
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr.add_data(encrypted_vc)
qr.make(fit=True)
img = qr.make_image(fill_color="black", back_color="white")
img.save("test/did_qr_code.png")  # Saving the QR code to a file

In [9]:
# Generate QR Code for the DID
qr_code_message = generate_qr_code(encrypted_vc)
print(qr_code_message)

QR code generated and saved as did_qr_code.png


In [10]:
from cryptography.fernet import Fernet

def verify_credential(encrypted_credential, key):
    try:
        f = Fernet(key)
        decrypted_data = f.decrypt(encrypted_credential).decode()
        credential = json.loads(decrypted_data)
        
        # Check for validity, integrity, authenticity
        if verify_signature(credential):
            print("Credential is valid and authentic.")
            return credential
        else:
            print("Invalid signature. Credential could be tampered with.")
    except Exception as e:
        print(f"Error during verification: {str(e)}")

def verify_signature(credential):
    # Dummy function for signature verification
    return True  # In real implementation, you would check the digital signature here

# Example usage:
# Assume `encrypted_credential` is what you retrieved and decrypted from the database after scanning the QR
# `key` is the Fernet key used for decryption
# decrypted_credential = verify_credential(encrypted_credential, key)


### Reading qr code

In [11]:
# !pip install pillow

In [12]:
# !pip install opencv-python

In [13]:
# !pip install pyzbar

In [36]:
import cv2
from pyzbar import pyzbar

In [37]:
trial_img= cv2.imread('test/did_qr_code.png')

In [38]:
qcd = cv2.QRCodeDetector()

In [39]:
data, bbox, _ = qcd.detectAndDecode(trial_img)

In [40]:
data

'gAAAAABmMqpy8DUgodmpXdWgZLRc92hCzR1P6I-5qqw3mc9Ck74qlo4F-4uvM_77D3NDUvEh41zGIG4GHM8iAy7fVO1VObg7CkETY604Y7w8XB9Zb4bLfqIgZ9kyFSUXNqRL91F8zPJY81v9goyBjUm1_6Ya37XjoRJf0bDFYcSS-rnXAlshUklPhvKn3kAt883O9anuAw9cM8pL88JsAOVDC7FKHnMVfwZiysSQp6FN-6JL6i2YxOULfzYq1B3amWQoxFc_3jdpDhgFE582xBWDdzw-bRw9D1HMsygLOzGU5hZUe_FC_kird5bERASEBBjMsY9VjsjTSbzE94sKZO9YFW-EQ3cIJPhyeYt21RFcr0B01onlqlFUYrU7SGlLI5i_dNHzIcHz_1TNstqSVRVFS98_BHOPgySw9wIkYW09Io9z9DiNvEx6kXc7gzr_RKSeFQ37309YOp-s6h8BFD3uSkLWWr5873_1pCQ6p1lxHw6EW8E1WmsdLVcNjCEUE-j6FbH-J6RHWRdJa-DEMQTa_f0lcEe03A=='

In [41]:
# Decrypting the VC for display
decrypted_vc = decrypt_data(data, key)
print("\nDecrypted Verifiable Credential:")
print(decrypted_vc) 


Decrypted Verifiable Credential:
{
  "context": "https://www.w3.org/2018/credentials/v1",
  "type": [
    "VerifiableCredential"
  ],
  "issuer": "did:example:975de2a3-e0e6-4775-9ea2-d758fe3a2277",
  "credentialSubject": {
    "id": "did:example:496d8f01-14bc-440d-b7cf-00e9895c01ef",
    "name": "John Doe",
    "studentId": "UNINotts-123456",
    "course": "Computer Science"
  }
}


In [ ]:
retval, decoded_info, points, straight_qrcode = qcd.detectAndDecodeMulti(trial_img)

In [ ]:
retval

True

In [ ]:
decoded_info

('gAAAAABmMqijUsYYHYbYf8vWH_kU0QRputEBoYCuEvqx4aSL2AfUsoH0MaCRxPk0Oi85mLoLoKppp7qGoNAgbbNgLW-8R_bXZX1FsufT8RdqwBsjDUv1p7fz0wHX6qNEMzkoeXXk4j_JyYi_LSrqmIYfGFuXMNXEfudg5Vly95t0aNfnoUm65Y7yyGj38_5Ci_9nLhpy5fehhzghNaghtBTQ6GrwIfcxQmFEWYXepY2tHLqTNDYeFvJe7CaINE29A-5AegmNgYtCLeiPijCcf2alRGtJv7u6aoRORQAVkinlIIGxEuwd9szrWd_TzNBNfIDWvdr0_XTZYY_7b638XCi5t56Bkkytv5x9klA7cSjIOy3gUjPBegcozH2DPq-OgBunROq4h4UBjGZmsksLDVTTdiaVidH_P7-gep9PDsZDwO1Dgcrex08VUgMDI-ZHc0a8L005p4MC3dylvAMnkn7DI2kO4CwQfTNJiZYCjcpgGDYUZYypxq0Rrm3K6YM6x2m7mSxtkzT6NC5UIkja3f40_82WzBNTMg==',)